# Environment Setup
## City Overview
Simulate a virtual city with a population of 500,000. The city is divided into 12 administration districts, each containing a number of sub-regions (cells). These sub-regions are randomly assigned as work locations, education facilities, social places, and residential areas.

## City Structure
* Total Cells: 1,000,000.
* Total Population: 500,000.
* Number of Administrative Districts: 12

## Location Distribution
Within the 1,000,000 cells, the distribution of different types of locations:
* Work Locations: Some cells will be randomly designated as work locations.
* Education Facilities: Some cells will be randomly designated as schools.
* Social Places: Additional cells will be randomly designated as social places, such as parks, cafes, shopping malls, etc.
* Residential Areas: The remaining cells will be designated as residential areas.

## Daily Time Point
The simulation will include three key time points each dat:
* Morning: Agents travel to work, school, or other daily activities
* Evening: Agents return home or visit social places
* Night: Agents stay at home resting

# Virus Description
The spread of the virus within the simulation is defined by several parameters, such as:
* Transmission Rate
* Latency Period
* Infection Duration: Recovery Rate, Death Rate
* ......

# Agent Description
## Agent Attributes
Each agent in the simulation will have the following attributes:

### 1. Age:
* Represents the age of the agent, affecting their behavior, occupation and risk of severe disease.

### 2. Occupation:
Defines the agents's primary daily activity and location. Occupations include student, worker, retiree, etc.

### 3. Health Status (SEIRD):
* Susceptible (S): The agent is healthy but can become infected
* Exposed (E): The agent has been exposed to the virus, but it is not yet infectious
* Infected (I): The agent is infected and can spread the virus
* Recovered (R): The agent has recovered from the infection and is immune
* Deceased (D): The agent has died from the infection.

### 4. Mask Wearing Status:
* Whether the agent is currently wearing a mask (yes/no)

### 5. Compilance Level with Lockdown Policies:
* Measures how strictly the agent follows lockdown and social distancing guidelines (as probabilities)

## Memory
Agents have the capability to remeber certain types of information:
* Health Status Changes:
  Agents can remeber their past health status, that interferes their choices for making traveling decisions, mask-wearing and compilance with lockdown policies
* Social Interactions:  
  Agents can record their social interactions, close contacts may lead to infection
* Environmental Information:
  Agents can retain information about the environment, such as the severity of outbreaks in specific areas, helping them make decisions about their movements and interacitons.

## Learning
Agents can adapt their behavior based on past experience and acquired knowledge:
* Behavioral Adjustment:
* Risk Adjustment:

## Agent Behavior Descriptions
### 1. Daily Activities:
    Agents follow a daily routine that involves going to work, school, or engaging in social activities. These routine vary depending on the agent's occupation and age group. The nature of activities change throughout different times of the day, include morning, evening and night.

### 2. Social Interactions:
    Agents engage in face-to-face interactions with other agents throughout the day. These interactions occur at work, school, social places and within residential areas.

### 3. Mask-Wearing Behavior:
    Agents decide whether to wear masks based on their health status, perceived risk, and compliance level.

### 4. Lockdown Policies:
    Agents' behavior is influenced by lockdown policies in place, such as movement restrictions and lockdowns.

## SIERD Model
* Susceptible: These agents have not yet been infected with the virus but are at risk of becoming infected  
* Exposed: These agents have been exposed to the virus and are infected, but are not yet infectious to others  
* Infected: These agents are currently infected and are capable of transmitting the virus to susceptible individuals    
* Recovered: These agents have recovered from the infection and are typically assumed to have immunity    
* Dead: These agents have died as a result of the infection

## Rules
### 1. Infection Spread Rules:
* When an infected agent comes into contact with a susceptible agent, there is a probability of virus transmission
* The probability of transmission may increase when a susceptible agent comes into contact with multiple infected agents
### 2. Social Interaction Rules:
* Agents’ social behavior may be influenced by their infection status and the surrounding environment. For example, during outbreaks, agents may be  more inclined to avoid crowded areas or maintain social distancing.
### 3. Government Intervention Rules:
* Government may implement control measures such as lockdowns, mask-wearing policies travel to slow down virus spread.

# Government Policies
* No Interventions
* Lockdown Only
* Mask Policy Only
* Combination of Lockdown and Mask Policy